<a href="https://colab.research.google.com/github/jinwoo3239/DeepLearning_study/blob/main/basic_torch_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
print(torch.__version__)

1.12.1+cu113


In [4]:
train_data = datasets.FashionMNIST(root='data', train=True, transform=ToTensor(), download=True)
test_data = datasets.FashionMNIST(root='data', train=True, transform=ToTensor(), download=True)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [12]:
batch_size = 64

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

len(train_dataloader)

937

In [13]:
samples = next(iter(train_dataloader))
samples[0].shape

torch.Size([64, 1, 28, 28])

In [19]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MyModel(nn.Module):
    def __init__(self, ):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) # 1, 28, 28 > 32, 28, 28
        self.pool = nn.MaxPool2d(2) # 32, 14, 14
        self.flatten = nn.Flatten() # 32 * 7 * 7
        self.dense1 = nn.Linear(32*14*14, 128)
        self.dense2 = nn.Linear(128, 10)

        # self.relu = nn.ReLU()
        # self.softmax = nn.Softmax()

        # self.layer1 = nn.Sequential(nn.Conv2d(), nn.ReLU(), nn.Maxpool2d)
        # self.flatten = nn.Flatten()
        # self.dense_layer = nn.Sequential(nn.Linear(), nn.ReLU(), nn.Linear(), nn.Softmax())

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = F.relu(self.dense1(x))
        output = F.softmax(self.dense2(x))
        return output


model = MyModel().to(DEVICE)

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-3)

In [21]:
def train_step(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader)

    for batch, (images, labels) in enumerate(dataloader):
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        pred = model(images)
        loss = loss_fn(pred, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 400 == 0:
            loss = loss.item()
            print(f'loss : {loss}')

In [22]:
def test_step(dataloader, model, loss_fn):

    model.eval()
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            pred = model(images)
            test_loss += loss_fn(pred, labels).item()
            pred_label = torch.argmax(pred, axis=1)
            pred_label = (pred_label == labels).type(torch.float).sum().item()
            correct += pred_label

    correct /= size
    print('Test step_')
    print(f'loss : {test_loss}, accuracy : {correct * 100 :.3f}')

In [23]:
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_step(train_dataloader, model, loss_fn, optimizer)
    test_step(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss : 2.301159143447876


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


loss : 2.3830254077911377
loss : 2.367400884628296
Test step_
loss : 2214.7626526355743, accuracy : 10.000
Epoch 2
-------------------------------
loss : 2.304900884628296
loss : 2.351775884628296
loss : 2.04858136177063
Test step_
loss : 1702.7285120487213, accuracy : 64.570
Epoch 3
-------------------------------
loss : 1.8017019033432007
loss : 1.7801421880722046
loss : 1.5930672883987427
Test step_
loss : 1498.2403010129929, accuracy : 86.803
Epoch 4
-------------------------------
loss : 1.581251621246338
loss : 1.5626448392868042
loss : 1.5596907138824463
Test step_
loss : 1468.143320798874, accuracy : 89.917
Epoch 5
-------------------------------
loss : 1.6046282052993774
loss : 1.6008471250534058
loss : 1.5684459209442139
Test step_
loss : 1457.6446833610535, accuracy : 90.935
Done!


In [25]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [26]:
model = MyModel()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>